In [1]:
import pandas as pd
akshit_df = './mlchallenge_set_2021.tsv'
akshit_valid = './mlchallenge_set_validation.tsv'
sam_df = 'C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_2021_edited.txt'
sam_valid = 'C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_validation.tsv'
sa_df = '/Users/shivankagrawal/Documents/ebay/mlchallenge_set_2021.tsv'
sa_valid = '/Users/shivankagrawal/Documents/ebay/mlchallenge_set_validation.tsv'
SA_valid=pd.read_table(akshit_valid,header=None)
SA_df=pd.read_table(akshit_df,header=None)
df=SA_df
valid=SA_valid
#df = pd.read_table(akshit_df)
#valid = pd.read_table(akshit_valid,sep='\t')
df.columns=['category','primary_image_url','All Links','Tags','index']
valid.columns=['ID', 'Group']

In [2]:
category5 = df.loc[df['category'] == 1]
len(category5)

501570

In [3]:
#create dictionary of groups with list of IDs in that group
dict = {}
total=0
for index,row in valid.iterrows():
    if row['Group'] in dict:
        dict[row['Group']].append(row['ID'])
    else:
        dict[row['Group']] = [row['ID']]
#list of groups with multiple IDs
matches = []
for key,value in dict.items():
    if len(value) > 1:
        total+=len(value)
        matches.append(key)
print(len(matches))
print(total)

990
2480


In [4]:
print(df.head())
print(valid.head())

   category                                  primary_image_url  \
0         2  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1         2  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2         2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3         2  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4         2  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                           All Links  \
0  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                                Tags  index  
0  (Brand:Shimano,US Shoe Size (Men's):4.5,Modifi...      0  
1  (Color:Gray/White,Country/Region of Manufactur...      1  
2  (Style:Cleats,Color:White Orange,US Shoe Size ...      2  
3  (Width:Medium (

In [5]:
#split links into list (for now only does first 100)
i = 0
for link in df['All Links'][0:100]:
    df['All Links'][i] = link.split(';')
    i+=1
print(df['All Links'][0:100])

<ipython-input-5-5e0f35be6386>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['All Links'][i] = link.split(';')


0     [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYY...
1     [https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPI...
2     [https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flI...
3     [http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcA...
4     [https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scs...
                            ...                        
95    [https://i.ebayimg.com/00/s/MTU5OVgxNTgx/z/vMA...
96    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/GoA...
97    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/Hm0...
98    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/VhA...
99    [https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/n4o...
Name: All Links, Length: 100, dtype: object


In [21]:
# df = category5
import re
def register_attributes(attribute, all_attributes):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        all_attributes.add(attribute[i][0])
    #print(f'atttribute is: {attribute}')
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return(attribute)

def map_attributes(attribute, num_attributes, index_to_attr):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    all_attributes_for_row = [None] * num_attributes
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        #print(f'index: {attr_to_index[attribute[i][0]]}')
        if len(attribute[i]) > 1:
            all_attributes_for_row[attr_to_index[attribute[i][0]]] = attribute[i][1]
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return all_attributes_for_row
m = 50000
all_attributes = set()
all_maps = []
for index,row in df[0:m].iterrows():
    register_attributes(row['Tags'], all_attributes)

all_attributes = list(all_attributes)
attr_to_index = {all_attributes[i]: i for i in range(len(all_attributes))}
#print(attr_to_index)
#print(f'numAttributes: {len(all_attributes)}')

for index,row in df[0:m].iterrows():
    all_maps.append(map_attributes(row['Tags'], len(all_attributes), attr_to_index))
#print(all_maps)


In [9]:
#dataframe with attribute values
categories = pd.DataFrame(all_maps)
print(categories)

       0     1     2     3     4     5     6     7     8     9    ...   429  \
0     None  None  None  None  None  None  None  None  None  None  ...  None   
1     None  None  None  None  None  None  None  None  None  None  ...  None   
2     None  None  None  None  None  None  None  None  None  None  ...  None   
3     None  None  None  None  None  None  None  None  None  None  ...  None   
4     None  None  None  None  None  None  None  None  None  None  ...  None   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
2995  None  None  None  None  None  None  None  None  None  None  ...  None   
2996  None  None  None  None  None  None  None  None  None  None  ...  None   
2997  None  None  None  None  None  None  None  None  None  None  ...  None   
2998  None  None  None  None  None  None  None  None  None  None  ...  None   
2999  None  None  None  None  None  None  None  None  None  None  ...  None   

       430   431   432   433   434   435   436   43

In [22]:
from sklearn.preprocessing import OneHotEncoder
x = OneHotEncoder().fit_transform(all_maps)
print(x)
print(type(x))

  (0, 0)	1.0
  (0, 2)	1.0
  (0, 4)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 26)	1.0
  (0, 27)	1.0
  (0, 29)	1.0
  (0, 30)	1.0
  (0, 31)	1.0
  (0, 33)	1.0
  (0, 35)	1.0
  (0, 37)	1.0
  (0, 39)	1.0
  (0, 41)	1.0
  (0, 42)	1.0
  (0, 43)	1.0
  (0, 44)	1.0
  :	:
  (49999, 68287)	1.0
  (49999, 68288)	1.0
  (49999, 68289)	1.0
  (49999, 68290)	1.0
  (49999, 68296)	1.0
  (49999, 68298)	1.0
  (49999, 68304)	1.0
  (49999, 68305)	1.0
  (49999, 68306)	1.0
  (49999, 68307)	1.0
  (49999, 68308)	1.0
  (49999, 68309)	1.0
  (49999, 68310)	1.0
  (49999, 68311)	1.0
  (49999, 68314)	1.0
  (49999, 68321)	1.0
  (49999, 68323)	1.0
  (49999, 68324)	1.0
  (49999, 68346)	1.0
  (49999, 68347)	1.0
  (49999, 68348)	1.0
  (49999, 68349)	1.0
  (49999, 68351)	1.0
  (49999, 68353)	1.0
  (49999, 68355)	1.0
<class 'scipy.sparse.csr.csr_matrix'>


In [37]:
"""
K-Means to pre-process data into smaller chunks before running agglomerative clustering on them because that is quadratic.
"""
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0,verbose=True, n_init=2).fit(x[:20000])

Initialization complete
Iteration 0, inertia 257548.0
Iteration 1, inertia 170089.3505034471
Iteration 2, inertia 166805.1933771165
Iteration 3, inertia 165597.7363711091
Iteration 4, inertia 164896.44604946204
Iteration 5, inertia 164671.48905171605
Iteration 6, inertia 164535.0551167738
Iteration 7, inertia 164302.23746608073
Iteration 8, inertia 163894.2821875912
Iteration 9, inertia 163625.11170008167
Iteration 10, inertia 163517.8283235627
Iteration 11, inertia 163485.3295080219
Iteration 12, inertia 163477.55125458544
Iteration 13, inertia 163475.71931548815
Iteration 14, inertia 163475.10911628767
Iteration 15, inertia 163474.67906088068
Iteration 16, inertia 163474.37842408242
Iteration 17, inertia 163474.19098511917
Iteration 18, inertia 163474.11533144282
Iteration 19, inertia 163474.05878266378
Iteration 20, inertia 163474.0132280555
Converged at iteration 20: strict convergence.
Initialization complete
Iteration 0, inertia 253100.0
Iteration 1, inertia 172007.97351980166
It

In [38]:
"""
View K-Means analysis
"""
count = {}
for label in kmeans.labels_:
    if label not in count:
        count[label] = 0
    count[label] += 1
print(count)
print(kmeans.inertia_)

{4: 3597, 7: 1736, 9: 3487, 6: 3542, 2: 1541, 5: 1336, 1: 1650, 3: 989, 8: 1290, 0: 832}
163062.43573854447


In [55]:
x_smol = x[0:3000]

In [56]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(compute_distances=True,compute_full_tree = True,distance_threshold = 1.5,n_clusters=None).fit(x_smol.todense())
print(clustering)

AgglomerativeClustering(compute_distances=True, compute_full_tree=True,
                        distance_threshold=1.5, n_clusters=None)


In [17]:
from sklearn.cluster import OPTICS
clustering2 = OPTICS(max_eps=.005,min_samples=2,cluster_method='dbscan', n_jobs=-1).fit(x.toarray())

KeyboardInterrupt: 

In [56]:
print("LABELS")
print(clustering.labels_)
print(len(clustering.labels_))
print(len(set(clustering.labels_)))
print("DISTAnCES")
print(clustering.distances_)
print("num connected components")
print(clustering.n_connected_components_)
# make this better
# make this work on the entire dataset
# fix nonetypes
# don't punish missing attributes, but punish conflicts. how do we encode this?

LABELS
[1950 1024  568 ...  977  307   83]
3000
2390
DISTAnCES
[ 0.          0.          0.         ... 39.53753098 50.13887074
 54.26601965]
num connected components
1


In [77]:
category2[0:1]

,category,primary_image_url,All Links,Tags,index
37976,5,https://i.ebayimg.com/00/s/MTIwMFgxMjAw/z/ec4A...,https://i.ebayimg.com/00/s/MTIwMFgxMjAw/z/ec4A...,"(Filters:HEPA,Model:T600),Country/Region of Ma...",38325


In [78]:
groups = {}
for i, label in enumerate(clustering.labels_):
    if label not in groups:
        groups[label] = []
    groups[label].append(i)
groups = {label: groups[label] for label in groups if len(groups[label]) > 1}
print(groups)
for label in groups:
    print(f'GROUP: {label}')
    for item in groups[label]:
        print(df['Tags'].iloc[[item]])
    print('-----------------')


{1024: [1, 21, 71, 72, 114, 117, 194, 255, 279, 311, 312, 361, 366, 431, 432, 434, 454, 512, 544, 563, 589, 655, 657, 660, 680, 694, 730, 748, 831, 868, 955, 956, 973, 1083, 1147, 1169, 1173, 1281, 1320, 1328, 1333, 1363, 1387, 1389, 1391, 1392, 1396, 1398, 1449, 1453, 1465, 1491, 1535, 1547, 1550, 1647, 1661, 1689, 1769, 1807, 1814, 1824, 1830, 1849, 1861, 1869, 1894, 1940, 1975, 1986, 2009, 2028, 2058, 2068, 2095, 2115, 2128, 2144, 2162, 2209, 2227, 2262, 2270, 2280, 2295, 2402, 2416, 2451, 2465, 2523, 2526, 2574, 2602, 2622, 2637, 2668, 2669, 2756, 2822, 2855, 2901, 2935, 2941, 2990], 568: [2, 2742, 2758], 371: [8, 490, 1322, 2430, 2777], 374: [13, 2670], 105: [15, 16, 124, 291, 292, 293, 2310, 2477], 160: [19, 1075], 186: [23, 1565], 8: [24, 29, 100, 2286, 2383, 2406], 209: [25, 2006], 1958: [26, 2223], 273: [30, 2225], 1030: [34, 1198, 1354], 68: [36, 2488, 2603, 2761], 185: [37, 50, 2276, 2350], 585: [42, 2736], 4: [49, 2318], 77: [53, 2699], 73: [54, 79], 22: [56, 176], 880: [74

Name: Tags, dtype: object
39932    (Model:PUCRC26B)
Name: Tags, dtype: object
40239    (Model:Dyson DC40)
Name: Tags, dtype: object
40478    (Model:2080-4)
Name: Tags, dtype: object
-----------------
GROUP: 163
38209    (Model:81L2A,MPN:81L2A,Brand:Bissell)
Name: Tags, dtype: object
38476    (MPN:81L2A,Model:81L2A,Brand:Bissell)
Name: Tags, dtype: object
-----------------
GROUP: 42
38214    (UPC:Does not apply,Brand:Dirt Devil,EAN:Does ...
Name: Tags, dtype: object
38360    (EAN:Does not apply,ISBN:Does not apply,MPN:Do...
Name: Tags, dtype: object
39540    (UPC:Does not apply,EAN:Does not apply,Brand:I...
Name: Tags, dtype: object
39823    (EAN:Does not apply,UPC:Does not apply,ISBN:Do...
Name: Tags, dtype: object
40404    (Brand:Alta Calidad,MPN:Does not apply,UPC:Doe...
Name: Tags, dtype: object
-----------------
GROUP: 439
38218    (Type:Robotic,MPN:PUCRC26B,Brand:Pyle)
Name: Tags, dtype: object
39919    (Brand:Pyle,MPN:PUCRC62,Type:Robotic)
Name: Tags, dtype: object
40017    (MPN:

Name: Tags, dtype: object
40343    (Model:v7 motorhead,Type:Stick,MPN:Does Not Ap...
Name: Tags, dtype: object
-----------------
GROUP: 415
38630    (Country/Region of Manufacture:China,MPN:Does ...
Name: Tags, dtype: object
39810    (Country/Region of Manufacture:China,MPN:Does ...
Name: Tags, dtype: object
40014    (Country/Region of Manufacture:China,Brand:Unb...
Name: Tags, dtype: object
40016    (Country/Region of Manufacture:China,MPN:Does ...
Name: Tags, dtype: object
-----------------
GROUP: 123
38632    (Country/Region of Manufacture:United States,U...
Name: Tags, dtype: object
40363    (ISBN:Does not apply,UPC:Does not apply,Brand:...
Name: Tags, dtype: object
-----------------
GROUP: 40
38639    (Type:Robotic,MPN:Does Not Apply,Brand:Roomba)
Name: Tags, dtype: object
39373    (Brand:Shark,MPN:Does Not Apply,Type:Robotic)
Name: Tags, dtype: object
40792    (Type:Robotic,MPN:Does Not Apply,Brand:Intertek)
Name: Tags, dtype: object
-----------------
GROUP: 210
38645    (Country

Name: Tags, dtype: object
-----------------
GROUP: 122
40047    (Brand:RIDGID,MPN:HD1401)
Name: Tags, dtype: object
40926    (Brand:RIDGID,MPN:Does Not Apply)
Name: Tags, dtype: object
-----------------
GROUP: 294
40049    (Brand:Oreck)
Name: Tags, dtype: object
40941    (Brand:Oreck)
Name: Tags, dtype: object
-----------------
GROUP: 246
40052    (Manufacturer Part Number:Does not apply,UPC:D...
Name: Tags, dtype: object
40092    (UPC:Does not apply,Manufacturer Part Number:D...
Name: Tags, dtype: object
-----------------
GROUP: 292
40074    (Brand:Roomba,Type:Robotic)
Name: Tags, dtype: object
40232    (Brand:Roomba,Type:Robotic)
Name: Tags, dtype: object
40258    (Type:Robotic,Brand:Roomba)
Name: Tags, dtype: object
-----------------
GROUP: 308
40080    (Brand:Shark,Type:Upright)
Name: Tags, dtype: object
40318    (Type:Upright,Brand:Shark)
Name: Tags, dtype: object
-----------------
GROUP: 127
40090    (Brand:Unbranded,Country/Region of Manufacture...
Name: Tags, dtype: object
4081

In [98]:
%matplotlib qt
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
import numpy as np
from PIL import Image
import requests
from io import BytesIO

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    #response = requests.get(url)
    #img = Image.open(BytesIO(response.content))
    link_labels = [df['Tags'][i] for i in clustering.labels_]
    dendrogram(linkage_matrix, labels = link_labels)
plot_dendrogram(clustering)
plt.show()

In [31]:
import re
from collections import Counter
freq=Counter()
attribute=[['']]*len(df)
trialrange=100
for x in range(trialrange):#range(int(len(df)/10)):#len(df)
    attribute[x]=df.iloc[x,3].lower()
    attribute[x] = re.sub(r'[()]','', attribute[x])
    attribute[x] = re.split(r',', attribute[x])
    attribute[x] = [a.split(':') for a in attribute[x]]
    freq+=Counter([i[0] for i in attribute[x]])
    tempdict={}
    for i in attribute[x]:

            try:
                tempdict[i[0]]=float(i[1])
            except:
                try:
                    tempdict[i[0]]=i[1]
                except:
                    pass
    attribute[x]=tempdict

df['seg']=attribute
#print(df['seg'])

In [ ]:
Brands=[]
Images=[]
print(trialrange)
for i in range(trialrange):
    try:
        #df['seg'].iloc[i]['brand']
        #df['primary_image_url'].iloc[i]
        #print(df['seg'].iloc[i]['brand'])
        #print(df['primary_image_url'].iloc[i])
        Brands.append(df['seg'].iloc[i]['brand'])
        Images.append(df['primary_image_url'].iloc[i])
    except:
        print('not possible at: ',i)

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np
'''
url = df['primary_image_url'][4]
response = requests.get(url)
#img = Image.open(BytesIO(response.content))
img = Image.open(requests.get(url, stream=True).raw)
img.show()
result = Image.new(img.mode, (1000, 550), (64,64,64))
result.paste(img, (0, 0))
result.show()
print(np.asarray(img).shape)
print(np.asarray(result).shape)
'''
image_array = []
max_height = 0
max_width = 0
for img in df['primary_image_url'][0:1000]:
    img = Image.open(BytesIO(response.content))
    if np.asarray(img).shape[1] > max_width:
        max_width = np.asarray(img).shape[1]
    if np.asarray(img).shape[0] > max_height:
        max_height = np.asarray(img).shape[0]

for img in df['primary_image_url'][0:1000]:
    img = Image.open(BytesIO(response.content))
    margin = Image.new(img.mode, (max_width, max_height), (64,64,64))
    margin.paste(img, (0, 0))
    image_array.append(np.asarray(margin))

In [ ]:
print(image_array[51].shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(400, 300, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

m = 100
train_images = data[0:m*.8]
test_images = data[m*.8:]
train_labels = labels[0:m*.8]
test_labels = labels[m*.8:]
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))